In [1]:
import numpy as np
import pandas as pd
import managePeakData as mpd

In [2]:
df = mpd.importPeak(1)

In [3]:
df.head()

,Ximf0,Ximf1,Ximf2,Yimf0,Yimf1,Yimf2,sticking,time_begin,time_end
0,4.00048,4.00048,4.00048,10.997140,53.246311,27.099370,False,90.0,94.0
1,4.00048,4.00048,4.00048,23.328407,42.661419,32.425791,False,94.0,98.0
2,4.00048,4.00048,4.00048,16.354136,67.408223,28.528204,False,98.0,102.0
3,4.00048,4.00048,4.00048,12.281024,25.186493,24.820231,False,102.0,106.0
4,4.00048,4.00048,4.00048,10.416508,26.581959,15.601763,False,106.0,110.0


In [6]:
from sklearn import tree
X = []
Y = []
for coil in [1,8,9,28,44,5,6,10,11,20,29,58,25,14,15,16,18]:
    df = mpd.importPeak(coil)
    Xi = zip(
            list(df['Ximf0']), list(df['Yimf0']), 
            list(df['Ximf1']), list(df['Yimf1']), 
            list(df['Ximf2']), list(df['Yimf2'])
            )
    Yi = list(df['sticking'])
    X += Xi
    Y += Yi
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
sum(Y)

47

In [7]:
for coil in range(88):
    try:
        df = mpd.importPeak(coil)
        X = zip(
                list(df['Ximf0']), list(df['Yimf0']), 
                list(df['Ximf1']), list(df['Yimf1']), 
                list(df['Ximf2']), list(df['Yimf2'])
                )
        Y = list(df['sticking'])
        print coil, sum(Y), sum(clf.predict(X))
    except:
        pass

0 0 4
1 0 0
2 8 1
3 14 1
4 6 2
5 0 0
6 0 0
7 27 26
8 7 7
9 1 1
10 0 0
11 0 0
12 0 1
13 7 1
14 0 0
15 0 0
16 0 0
17 0 1
18 0 0
19 0 1
20 11 11
21 7 1
22 0 1
23 0 4
24 0 0
25 0 0
26 0 0
27 0 6
28 13 13
29 13 13
30 0 3
32 0 0
33 0 3
34 4 4
35 0 1
36 0 3
37 5 1
38 14 9
39 0 1
40 12 4
41 0 4
42 0 0
43 3 2
44 2 2
45 0 5
46 0 9
47 0 2
48 0 1
49 29 3
50 10 2
51 0 6
52 4 3
53 8 2
54 0 1
55 0 4
56 0 5
57 0 6
58 0 0
59 0 1
60 0 0
61 0 1
62 10 7
63 0 2
64 0 5
65 18 2
66 9 1
67 0 1
68 0 7
69 0 1
70 0 4
71 8 5
72 16 5
73 0 2
74 12 6
75 0 1
76 0 0
77 0 19
78 29 4
79 15 1
80 15 3
81 27 5
82 17 2
83 12 2
84 0 3
85 0 5
86 0 5
87 0 0
